In [1]:
from selenium import webdriver 
#from selenium.webdriver.chrome.options import Options
import time 

In [13]:
#options = Options()

In [14]:
#url="https://coinmarketcap.com/"


In [15]:
#driver = webdriver.Chrome(options=options)
#try:
    #driver.get(url) #HTTP GET request - requests.get(url)
    #time.sleep(10)
#finally:
    #driver.quit()
!python.exe -m pip install --upgrade pip
!pip install selenium webdriver-manager


In [29]:
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup

from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC



```html
<span class="sc-d1ede7e3-0 fsQm base-text">$0.004871</span>
<p color="red" data-change="down" font-size="1" data-sensors-click="true" class="sc-71024e3e-0 sc-58c82cf9-1 ihXFUo iPawMI">14.75%&nbsp;(1d)</p>

<dd class="sc-d1ede7e3-0 hPHvUM base-text">$46,526,472</dd>

//market cap rank
<span class="text slider-value rank-value">#665</span>
//volume
<dd class="sc-d1ede7e3-0 hPHvUM base-text">$13,138,603</dd>
//volume_rank
<span class="text slider-value rank-value">#377</span>
//volume_change
<dd class="sc-d1ede7e3-0 hPHvUM base-text">29.32%</dd></div>
//circulating_supply
<dd class="sc-d1ede7e3-0 hPHvUM base-text">9,663,955,990 DUKO</dd>
//total_supply
<dd class="sc-d1ede7e3-0 hPHvUM base-text">9,999,609,598 DUKO</dd>
//diluted_market_cap
<dd class="sc-d1ede7e3-0 hPHvUM base-text">$48,142,454</dd>
//contracts 
//name 
<span font-weight="var(--c-font-weight-400)" font-size="var(--c-font-size-75)" color="text" data-sensors-click="true" 
    class="sc-71024e3e-0 dEZnuB">Solana:&nbsp;</span>
//address
<span font-weight="var(--c-font-weight-500)" font-size="var(--c-font-size-75)" class="sc-71024e3e-0 eESYbg address" color="text" data-sensors-click="true">HLptm5...2G7rf9&nbsp;</span>
//websites 
<span class="sc-d1ede7e3-0 fTGacL base-text" data-role="title">Official links</span>
//name
<a rel="nofollow noopener" href="https://github.com/ethereum/wiki/wiki/White-Paper" target="_blank">Whitepaper</a>
<a rel="nofollow noopener" href="https://github.com/ethereum/go-ethereum" target="_blank">GitHub</a>
//social links
<span class="sc-d1ede7e3-0 fTGacL base-text" data-role="title">Socials</span>
//name
<a rel="nofollow noopener" href="https://twitter.com/ethereum" target="_blank">Twitter</a>
<a rel="nofollow noopener" href="https://reddit.com/r/ethereum" target="_blank">Reddit</a>
```


In [130]:
from selenium.webdriver.support import expected_conditions as EC
class CoinMarketCapScraper:
    def __init__(self, coin):
        self.coin = coin
        self.url = f"https://coinmarketcap.com/currencies/{coin}/"
        self.driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))

    
    def get_coin_data(self):
        self.driver.get(self.url)
        self.driver.implicitly_wait(10) 
       
        try:
            # Wait for the price element to be loaded in the DOM
            WebDriverWait(self.driver, 20).until(
                EC.presence_of_element_located((By.CSS_SELECTOR, "span.sc-d1ede7e3-0.fsQm.base-text"))
            )
        except Exception as e:
            self.driver.quit()
            return None
        html_content = self.driver.page_source
        soup = BeautifulSoup(html_content, "html.parser")

        data = {}

        # Extract price
        try:
            price_element = soup.select_one('span.sc-d1ede7e3-0.fsQm.base-text')
            data['price'] = price_element.text if price_element else None
        except Exception as e:
            data['price'] = None
            
         # Extract price change
        try:
            price_change_element = soup.select_one('p[data-change="down"].sc-71024e3e-0.sc-58c82cf9-1.ihXFUo.iPawMI')
            if price_change_element:
                # Split the text content by "%" and take the first part
                price_change_text = price_change_element.text
                data['price_change'] = price_change_text.split("%")[0]
            else:
                data['price_change'] = None
        except Exception as e:
            data['price_change'] = None
            
      # Find the market cap rank and volume rank

        try:
            rank_elements = soup.find_all('span', class_='text slider-value rank-value')
            if rank_elements:
                data['market_cap_rank'] = rank_elements[0].text.strip()
                data['volume_rank'] = rank_elements[1].text.strip() if len(rank_elements) > 1 else None
            else:
                market_cap_rank = None
                volume_rank = None
        except Exception as e:
            print(f"Error occurred while extracting rank values: {e}")
            market_cap_rank = None
            volume_rank = None

        #volume and market_cap
        try:
            volume = soup.find_all('dd', class_='sc-d1ede7e3-0 hPHvUM base-text')
            if volume:
                # Extract the text content of the element
                data['market_cap']= volume[0].text.strip().split('%')[1]
                data['volume']=volume[1].text.strip().split('%')[1] if len(rank_elements) > 1 else None
                data['volume_change']=volume[2].text.strip()
                data['circulating_supply']=volume[3].text.strip()
                data['total_supply']=volume[5].text.strip()
                data['diluted_market_cap']=volume[6].text.strip()
            else:
                data['volume'] = None
                data['market_cap'] = None
                data['volume_change']=None
                data['circulating_supply']=None
                data['total_supply']=None
                data['diluted_market_cap']=None
        except Exception as e:
            print(f"Error occurred while extracting rank values: {e}")
            data['volume'] = None
            data['market_cap'] = None
            data['volume_change']=None
            data['circulating_supply']=None
            data['total_supply']=None
            data['diluted_market_cap']=None
        #extract contracts 
        try:
            contracts_elements = soup.find_all('span', class_='sc-71024e3e-0 dEZnuB')
            addresses_elements = soup.find_all('span', class_='sc-71024e3e-0 eESYbg address')
    
            contracts = []
            for contract_element, address_element in zip(contracts_elements, addresses_elements):
                name = contract_element.text.strip().split(':')[0]  # Extract the name
                address = address_element.text.strip()             # Extract the address
                contracts.append({'name': name, 'address': address})

            data['contracts'] = contracts
        except Exception as e:
            print(f"Error occurred while extracting contract values: {e}")
            data['contracts'] = []
    
        
        # Extract official links and socials
        try:
            links_section_elements = soup.find_all('div', class_='sc-d1ede7e3-0 jTYLCR')
        
            for section_element in links_section_elements:
                # Check if the section contains official links or socials
                title_element = section_element.find('span', class_='base-text', attrs={'data-role': 'title'})
                if title_element:
                    section_title = title_element.text.strip()
                    if section_title == 'Official links':
                        # Extract official links
                        official_links_elements = section_element.find_all('a', rel='nofollow noopener', target='_blank')
                        official_links = [{'name': link.text.strip(), 'link': link['href']} for link in official_links_elements]
                        data['official_links'] = official_links
                    elif section_title == 'Socials':
                        # Extract socials
                        socials_elements = section_element.find_all('a', rel='nofollow noopener', target='_blank')
                        socials = [{'name': social.text.strip(), 'url': social['href']} for social in socials_elements]
                        data['socials'] = socials
        except Exception as e:
            data['official_links'] = []
            data['socials'] = []
        return data
        
    def close(self):
        self.driver.quit()


In [131]:
scraper = CoinMarketCapScraper('Ethereum')
data = scraper.get_coin_data()
scraper.close()
print(data) 

{'price': '$3,684.95', 'price_change': '3.33', 'market_cap_rank': '#2', 'volume_rank': '#3', 'market_cap': '$442,710,548,093', 'volume': '$16,465,467,133', 'volume_change': '3.72%', 'circulating_supply': '120,147,950 ETH', 'total_supply': '∞', 'diluted_market_cap': '$442,710,548,093', 'contracts': [{'name': 'BNB Smart Chain (BEP20)', 'address': '0x2170...f933f8'}], 'official_links': [{'name': 'Whitepaper', 'link': 'https://github.com/ethereum/wiki/wiki/White-Paper'}, {'name': 'GitHub', 'link': 'https://github.com/ethereum/go-ethereum'}], 'socials': [{'name': '𝕏Twitter', 'url': 'https://twitter.com/ethereum'}, {'name': 'Reddit', 'url': 'https://reddit.com/r/ethereum'}, {'name': 'Chat', 'url': 'https://gitter.im/orgs/ethereum/rooms'}]}


In [132]:
!jupyter nbconvert --to script coinmarketcap.ipynb

[NbConvertApp] Converting notebook coinmarketcap.ipynb to script
[NbConvertApp] Writing 8865 bytes to coinmarketcap.py
